This homework is based off a notebook. 
1. You will alter this as need for your results / questions. 
2. When you've successfully run them on a colab, download as an .ipynb and submit on the LMS.

Make certain you are on a GPU Runtime first, by going to Runtime and selecting "Change Runtime Type", and then choosing Hardware Accelerator as GPU.

In this homework, you will build an emotion classifier based on a Huggingface emotions dataset.

You will need to install 🤗 Transformers, numpy and 🤗 Datasets. Run the following three cells.

In [19]:
pip install torch torchvision

     |████████████████████████████████| 120.6 MB 10.0 kB/s 
     |████████████████████████████████| 13.1 MB 1.5 MB/s 
     |████████████████████████████████| 2.2 MB 1.3 MB/s 
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [29]:
pip install optuna

     |████████████████████████████████| 287 kB 782 kB/s 
     |████████████████████████████████| 159 kB 2.4 MB/s 
     |████████████████████████████████| 1.2 MB 2.4 MB/s 
     |████████████████████████████████| 80 kB 2.3 MB/s 
     |████████████████████████████████| 75 kB 1.9 MB/s 
     |████████████████████████████████| 133 kB 4.8 MB/s 
     |████████████████████████████████| 259 kB 3.6 MB/s 
     |████████████████████████████████| 106 kB 2.5 MB/s 
     |████████████████████████████████| 49 kB 1.7 MB/s 
     |████████████████████████████████| 49 kB 2.1 MB/s 
Using legacy 'setup.py install' for pyperclip, since package 'wheel' is not installed.
    Running setup.py install for pyperclip ... done
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install sklearn

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install datasets==1.3.0

     |████████████████████████████████| 181 kB 798 kB/s 
     |████████████████████████████████| 81 kB 2.8 MB/s 
     |████████████████████████████████| 14.2 MB 776 kB/s 
     |████████████████████████████████| 16.1 MB 1.2 MB/s 
     |████████████████████████████████| 64 kB 938 kB/s 
     |████████████████████████████████| 103 kB 1.0 MB/s 
     |████████████████████████████████| 126 kB 845 kB/s 
     |████████████████████████████████| 69 kB 1.1 MB/s 
     |████████████████████████████████| 10.7 MB 715 kB/s 
     |████████████████████████████████| 510 kB 754 kB/s 
Using legacy 'setup.py install' for xxhash, since package 'wheel' is not installed.
    Running setup.py install for xxhash ... done
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [15]:
pip install transformers

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install numpy==1.20.1

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
from datasets import load_dataset
emotions_dataset = load_dataset('go_emotions', 'simplified')

Reusing dataset go_emotions (/Users/sarimkhans/.cache/huggingface/datasets/go_emotions/simplified/0.0.0/ef1c18ea192c771555f1e0d638889dd5f1896255782c57c6a0b934d5f94f779e)


To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [4]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [5]:
# For simplicity, train model with single label for item.
restricted_dataset = emotions_dataset.filter(lambda x: len(x["labels"]) == 1 and 27 not in x['labels'])
print(len(restricted_dataset['train']))
show_random_elements(restricted_dataset['train'])

Loading cached processed dataset at /Users/sarimkhans/.cache/huggingface/datasets/go_emotions/simplified/0.0.0/ef1c18ea192c771555f1e0d638889dd5f1896255782c57c6a0b934d5f94f779e/cache-30532e9cc7b8a9d6.arrow
Loading cached processed dataset at /Users/sarimkhans/.cache/huggingface/datasets/go_emotions/simplified/0.0.0/ef1c18ea192c771555f1e0d638889dd5f1896255782c57c6a0b934d5f94f779e/cache-d93795d3697d275e.arrow
Loading cached processed dataset at /Users/sarimkhans/.cache/huggingface/datasets/go_emotions/simplified/0.0.0/ef1c18ea192c771555f1e0d638889dd5f1896255782c57c6a0b934d5f94f779e/cache-9832485185737d89.arrow
23485


,id,labels,text
0,ef5fnn9,[17],Glad they made it..
1,eeyfzvw,[15],Thank you for commenting exactly what I was going to comment and saving me the trouble!
2,edw9jvd,[0],"Real nice, swearing up a storm right in front of your kids."
3,eebeui2,[10],"No, it was believed that the baby absorbed maternal traits because the mother was the ""soil"" the ""seed"" grew in."
4,eehewzl,[9],“Regain the fan base”. How immature do you have to be to believe that everyone agrees with you.
5,ed9y743,[20],"Yeah, hopefully the lesson learned from this is that our level of airport security is stupid and unnecessary and not that [NAME] is risking another 9/11"
6,ed62fhl,[4],"Yup, I'm sick with a mild cold"
7,ed2zfwd,[6],Why is it dumb? It's not like once people get married they believe in a society that looks down on premarital sex. Maybe you missed my point.
8,edc2z55,[0],Well good...opera is often beautiful..though it retains the stink of the upper classes.. I cant help but deride art that caters to high society to stay alive.
9,eeowi6x,[5],It may get you pregnant so just make sure you eat all her birth control before hand.



The emotions are provided as numeric labels. These are the actual orderings, beginning at 0 for admiration:

In [6]:
labels = ["admiration", "amusement", "anger", "annoyance", "approval", "caring", 
          "confusion", "curiosity", "desire", "disappointment", "disapproval",
          "disgust", "embarrassment", "excitement", "fear", "gratitude", "grief",
          "joy", "love", "nervousness", "optimism", "pride", "realization",
          "relief", "remorse", "sadness", "surprise", "neutral"]
index_to_labels = {index: label for index, label in enumerate(labels)}
  

## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [47]:
from transformers import AutoConfig, AutoModelForSequenceClassification, AutoTokenizer

model_name = 'distilbert-base-uncased'
model_init= AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=27)
#config = AutoConfig.from_pretrained(model_name, num_labels=27)
#model = AutoModelForSequenceClassification.from_pretrained(config)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

We pass along `use_fast=True` to the call above to use one of the fast tokenizers (backed by Rust) from the 🤗 Tokenizers library. Those fast tokenizers are available for almost all models, but if you got an error with the previous call, remove that argument.

In [48]:
def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True)

def convert_labels_to_int(example):
    example['labels'] = example['labels'][0]
    return example

encoded_dataset = restricted_dataset.map(preprocess_function, batched=True)
encoded_dataset = encoded_dataset.map(convert_labels_to_int)

Loading cached processed dataset at /Users/sarimkhans/.cache/huggingface/datasets/go_emotions/simplified/0.0.0/ef1c18ea192c771555f1e0d638889dd5f1896255782c57c6a0b934d5f94f779e/cache-cc7fe283fbd8c8d9.arrow
Loading cached processed dataset at /Users/sarimkhans/.cache/huggingface/datasets/go_emotions/simplified/0.0.0/ef1c18ea192c771555f1e0d638889dd5f1896255782c57c6a0b934d5f94f779e/cache-4caeb60e4efe21b3.arrow
Loading cached processed dataset at /Users/sarimkhans/.cache/huggingface/datasets/go_emotions/simplified/0.0.0/ef1c18ea192c771555f1e0d638889dd5f1896255782c57c6a0b934d5f94f779e/cache-83caf1489f390fdb.arrow
Loading cached processed dataset at /Users/sarimkhans/.cache/huggingface/datasets/go_emotions/simplified/0.0.0/ef1c18ea192c771555f1e0d638889dd5f1896255782c57c6a0b934d5f94f779e/cache-adf2e41c49e2c59e.arrow
Loading cached processed dataset at /Users/sarimkhans/.cache/huggingface/datasets/go_emotions/simplified/0.0.0/ef1c18ea192c771555f1e0d638889dd5f1896255782c57c6a0b934d5f94f779e/cach

## Fine-tuning the model

To instantiate a `Trainer`, we will need to define two more things. The most important is the [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [49]:
from transformers import Trainer, TrainingArguments
metric_name = "accuracy"

args = TrainingArguments(
    "test-emotions",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
)

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the notebook and customize the number of epochs for training, as well as the weight decay. Since the best model might not be the one at the end of training, we ask the `Trainer` to load the best model it saved (according to `metric_name`) at the end of training.

In [50]:
metric = datasets.load_metric('accuracy')
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

Then we just need to pass all of this along with our datasets to the `Trainer`:

In [51]:

validation_key = "validation"
trainer = Trainer(
    model_init,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

You might wonder why we pass along the `tokenizer` when we already preprocessed our data. This is because we will use it once last time to make all the samples we gather the same length by applying padding, which requires knowing the model's preferences regarding padding (to the left or right? with which token?). The `tokenizer` has a pad method that will do all of this right for us, and the `Trainer` will use it. You can customize this part by defining and passing your own `data_collator` which will receive the samples like the dictionaries seen above and will need to return a dictionary of tensors.

We can now finetune our model by just calling the `train` method:

In [52]:
import numpy as np
trainer.train()

9, 15.77s/it]

 71%|███████▏  | 656/920 [149:28:14<1:10:14, 15.96s/it]

 71%|███████▏  | 657/920 [149:28:30<1:10:04, 15.99s/it]

 72%|███████▏  | 658/920 [149:28:46<1:08:53, 15.78s/it]

 72%|███████▏  | 659/920 [149:29:03<1:10:39, 16.25s/it]

 72%|███████▏  | 660/920 [149:29:18<1:08:10, 15.73s/it]

 72%|███████▏  | 661/920 [149:29:32<1:06:20, 15.37s/it]

 72%|███████▏  | 662/920 [149:29:47<1:05:32, 15.24s/it]

 72%|███████▏  | 663/920 [149:30:03<1:06:38, 15.56s/it]

 72%|███████▏  | 664/920 [149:30:18<1:05:54, 15.45s/it]

 72%|███████▏  | 665/920 [149:30:33<1:04:16, 15.12s/it]

 72%|███████▏  | 666/920 [149:30:48<1:03:47, 15.07s/it]

 72%|███████▎  | 667/920 [149:31:03<1:03:42, 15.11s/it]

 73%|███████▎  | 668/920 [149:31:18<1:02:56, 14.99s/it]

 73%|███████▎  | 669/920 [149:31:32<1:01:59, 14.82s/it]

 73%|███████▎  | 670/920 [149:31:48<1:03:40, 15.28s/it]

 73%|███████▎  | 671/920 [149:32:03<1:03:03, 15.20s/it]

 73%|███████▎  | 672/920 [149:32:20<1:03:54, 15.46s/it]

 73%|███████▎  |

TrainOutput(global_step=920, training_loss=1.5183068980341372, metrics={'train_runtime': 542380.5423, 'train_samples_per_second': 0.002, 'epoch': 5.0})

We can check with the `evaluate` method that our `Trainer` did reload the best model properly (if it was not the last one). We also run a sample prediction to demonstrate the API:

In [1]:
trainer.evaluate()

NameError: name 'trainer' is not defined

In [ ]:
prepared_input = tokenizer.prepare_seq2seq_batch(["I am very sad"], return_tensors='pt')
model = model.to('cpu')
model.eval()
model_output = model(**prepared_input)
prediction = np.argmax(model_output.logits[0].detach().numpy())
index_to_labels[prediction]

# Your work

Qs 1. Can you report how accuracy changes with varying the hypyerparameters defined above? How does accuracy change with increasing number of epochs? Learning rate? Mention your final choice of parameters and graph how they impact accuracy. 

Qs 2. Try any 2 other models for sequence classification supported by Huggingface and repeat Qs 1 for these. You can find a full list of these [here](https://huggingface.co/transformers/model_doc/auto.html#automodelforsequenceclassification). (For those of you working with text for your project, you may also find several other useful models for Seq2Seq modeling, question answering, etc.)

Qs 3. Wrangle / change the data so that you can instead build a "positive" emotion vs "negative" emotions classifier, by combining labels. For any emotions that you feel are too neutral, you may drop them from evaluation. Can you now report the accuracy of a classifer on this data? You may use one of the model architectures from (1) or (2)

Qs 4. Export models from qs 1 and 3, and build a FastAPI around it, as discussed in class, which returns the most likely emotion found. (You may need to look up Huggingface documentation on how to save and load models). In particular, you should support two API endpoints - "detect_emotion_binary" and "detect_emotion_full" which supports each model.

Qs 5. Finally, can you dockerize this? You do NOT need to deploy this to the cloud.

Submission instructions: Zip your compiled .ipynb notebook, your fast_api code and your dockerfile, and submit to LMS as one file. You should confirm that your .ipynb can be imported to colab and run in sequence. All your results for Qs 1-3 should be reported in the notebook itself.